In [ ]:
import dnaFit
import dnaFit.viewer.viewer as app
from pathlib import Path

# 1. Parse filepaths and create Viewer instance
The viewer class contains both the data instances (design, map, model) and the selection functionality.
On creating the linkage between these fit and design is computed. this step might require a couple of seconds, depending on the design size.

In [ ]:
# specify working directory and design name
wd = "/Users/name/data/"
name = "design"

json = Path(wd +name + ".json")         # caDNAno design file
psf = Path(wd + name + ".psf")          # coordinate or trajectory file
coor = Path(wd + name + "-last.pdb")    # coordinate or trajectory file
mrc = Path(wd + name + ".mrc")          # cryo-EM map
seq = Path(wd + name + ".seq")          # scaffold sequence file


In [ ]:
viewer = app.Viewer(conf=coor, top=psf, mrc=mrc, json=json, seq=seq, is_mrdna=True)


# 2. Select subset of the atomic model
the viewer class supports different selection options:
* design-specific selection: subset specified by helix-id and base-position as in caDNAno designfile.
    * using selection widget -> option A
    * manual selection -> option B
* fit-specific selection: subset directly by MDAnalysis atomselection functionalities
    * example strand-specific selection: -> option C
    
    
In the end, all selections are translated into a selection of atom, stored as an MDAnalysis AtomGroup.   


### Option A: selection with widget 

#### NOTE: requires installed and activated ipywidgets

selection: selected helices (clicked) are displayed in dark grey. the "base" slider selects the base-position range.
context: slider for the area surrounding a selected atom for mrc-cropping (in Å)

---

the content of the widget is evaluated by executing

``helixandbase, context = viewer.eval_sliders(*sliders)`` 
which returns a tuple containing two lists([helix-ids],[base_ids]) and the context value

the list-tuple can be translated into an AtomGroup by executing:
``atoms_selection, color_dict = viewer.select_by_helixandbase(*helixandbase)``

the method returns the AtomGroup ``atoms_selection`` 

---

(it also returns the dictionary ``color_dict`` containing information about staple coloring in the .json file, that can be used to color staples in the ngl_view widget)

In [ ]:
sliders = viewer.select_widget()

In [ ]:
helixandbase, context = viewer.eval_sliders(*sliders)
atoms_selection, color_dict = viewer.select_by_helixandbase(*helixandbase)
print("helix an dbase:", helixandbase)
print("number of selected bases:", len(atoms_selection.residues))


### Option B: manual selection
``viewer.select_by_helixandbase(helices, base)`` can also be called directly using two lists, specifying the helix-ids and base positions.

the method can also be applied to multiple subsets of the data, as MDAnalysis AtomGroups can be combined to realize more complex selections. (more details see option C)

In [ ]:
## example Option B:
helix_selection = [1,2,4,5]
baseposition_selection = range(90,100)
atoms_selection, color_dict = viewer.select_by_helixandbase(helix_selection, baseposition_selection)

### Option C: Fit secific selection
generate atomselection using mdAnalysis. 
the MDAnalysis baseclass is available via ``viewer.linker.fit.u``

documentation:(https://www.mdanalysis.org/docs/documentation_pages/selections.html)

NOTES:
* AtomGroups are combinable using set-syntax (f.e. selection = selectionA + selectionB)
* AtomGroups can be inverted by subtraction them from the full system accessible via .universe.atoms attribute. this is usefull for creating masks for multibody and local scanning refinement processes.
* (design-specific staple coloring is not compatible with this option)

In [ ]:
## example Option C: all double-stranded segments with staples beginning with the sequence ATCG

""" either use MDanalysis directly to f.i.:
        select all staples beginning with a specific sequence,
        via the intersection of all staples with the correct base condition
        start with all segments and filter with additional conditions
""" 
ATCG = viewer.u.segments
for idx, X in enumerate(["ADE", "THY", "CYT", "GUA"]):
    ATCG = ATCG.segments & viewer.u.select_atoms("resname {} and resid {}".format(X, idx+1)).segments
atoms_staple  = ATCG.segments.atoms

""" or use custom selection methods to f.i.:
        pick all paired scaffold bases, while excluding Hydrogen atoms
"""
atoms_ds = viewer.select_ds_dna()
atoms_sc = viewer.select_scaffold(atoms=atoms_ds)
atoms_selection = viewer.select_without_H(atoms=atoms_sc)

# 3. Cropping and Zoning 
the generated AtomGroup can be used for
* generating a pdb of te subset represented by the AtomGroup
* creating a new mrc-map that only contains data in the vicinity (context) of the selected atoms 

---
create subset MRC-file

``viewer.writemrc(atoms_selection, wd, name, [context=context, cut_box=True])``

cut_box: if `True`, remove zero-padding from volume

---

create subset pdb-file (chimeraX compatible)

``viewer.writepdb(atoms_selection, wd, name, [single_frame=True, frame=-1, chimeraX=True])``

single_frame: if `False`, create multiframe pdb for full trajectory (multiframe can take a couple of minutes)

frame: frame index to be saved, default -1 = last (only for .dcd file as coordinate-file)\

as_cif: if `True`, also create mmcif coordinate file.

In [ ]:
#choose output name
name_out = "subset_selection"

In [ ]:
#create subset MRC-file
viewer.write_mrc(atoms_selection, name_out, context=context, cut_box=False)

In [ ]:
#create subset pdb-file (chimeraX compatible)
viewer.write_pdb(atoms_selection, name_out, single_frame=True, frame=-1, as_cif=True)